In [1]:
import sys
import getpass
sys.path.append('..')

In [2]:
import pandas as pd
import os
import re
import time
from sqlalchemy import create_engine, text, inspect, MetaData, Table, Column, Integer, Float, String, Text, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from geoalchemy2 import Geography, WKTElement
from FlowHandler import FlowHandler
from Akvo import Flow

In [3]:
PSQL_USER = getpass.getuser()
PSQL_PWD = os.environ['KEYCLOAK_PWD']
PSQL_DB = 'uduma'

In [4]:
instanceURI = 'angkorsalad'
requestURI = 'https://api.akvo.org/flow/orgs/' + instanceURI
surveysUrl = []

In [5]:
engine = create_engine("postgresql://{}:{}@localhost/{}".format(PSQL_USER, PSQL_PWD, PSQL_DB))
session = sessionmaker(bind=engine)()
Base = declarative_base()

## Helpers

In [6]:
then = time.time()

In [7]:
def getTime():
    now = time.time()
    decimal_time = int(( now - then)) / 60
    integer_time = int(decimal_time)
    seconds_time = decimal_time - integer_time
    seconds_time = int(60 * seconds_time)
    return '{} Minutes, {} Seconds'.format(integer_time, seconds_time)

## Classes

In [8]:
class Surveys(Base):
    
    __tablename__ = "survey"
    
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    registration_id = Column(Integer, nullable=True)
    
    def __init__(self, data):
        self.id = int(data['id'])
        self.name = data['name']
        self.registration_id = int(data['registrationFormId'])
        
    def __repr__(self):
        return "<Survey(id={}, name={}, registration_id={})>".format(
            self.id, self.name, self.registration_id)

class Forms(Base):
    
    __tablename__ = "form"
    
    id = Column(Integer, primary_key=True)
    survey_id = Column(Integer)
    name = Column(Text)
    
    def __init__(self, data):
        self.id = int(data['id'])
        self.survey_id = int(data['survey_id'])
        self.name = data['name']
    
    def __repr__(self):
        return "<Forms(id={}, survey_id={}, name={})>".format(
            self.id, self.survey_id, self.name)
        
class QuestionGroups(Base):
    
    __tablename__ = "question_group"
    
    id = Column(Integer, primary_key=True)
    form_id = Column(Integer)
    repeat = Column(Boolean)
    name = Column(Text)
    
    def __init__(self, data):
        self.id = int(data['id'])
        self.form_id = int(data['form_id'])
        self.repeat = data['repeatable']
        self.name = data['name']
    
    def __repr__(self):
        return "<QuestionGroups(id={}, form_id={}, repeat={}, name={})>".format(
            self.id, self.form_id, self.repeat, self.name)
        
class Questions(Base):
    
    __tablename__ = "question"
    
    id = Column(Integer, primary_key=True)
    form_id = Column(Integer)
    question_group_id = Column(Integer)
    name = Column(Text)
    type = Column(Text)
    
    def __init__(self, data):
        self.id = int(data['id'])
        self.form_id = int(data['form_id'])
        self.question_group_id = int(data['group_id'])
        self.name = data['name']
        self.type = data['type']
    
    def __repr__(self):
        return "<Questions(id={}, form_id={}, question_group_id={}, name={}, type={})>".format(
            self.id, self.form_id, self.question_group_id, self.name, self.type)

In [9]:
def writeData(input_data, info):
    try:
        session.add(input_data)
        print('INSERTING {}: {}'.format(info['name']))
        session.commit()
    except:
        print('ERROR: ABORTING {}'.format(info['name']))

## Main Function

In [10]:
def getFolders(items):
    print('GETTING FOLDER LIST: ' + getTime())
    for folder in items['folders']:
        try:
            surveysUrl.append(folder['surveysUrl'])
            childs = Flow.getResponse(folder['foldersUrl'])
            getFolders(childs)
        except:
            pass
    print('FOLDER IS POPULATED: ' + getTime())
        
parents = Flow.getResponse(requestURI + '/folders')
getFolders(parents)

GETTING FOLDER LIST: 0 Minutes, 8 Seconds
GETTING FOLDER LIST: 0 Minutes, 11 Seconds
GETTING FOLDER LIST: 0 Minutes, 16 Seconds
GETTING FOLDER LIST: 0 Minutes, 19 Seconds
GETTING FOLDER LIST: 0 Minutes, 23 Seconds
GETTING FOLDER LIST: 0 Minutes, 28 Seconds
GETTING FOLDER LIST: 0 Minutes, 32 Seconds
GETTING FOLDER LIST: 0 Minutes, 34 Seconds
GETTING FOLDER LIST: 0 Minutes, 38 Seconds
GETTING FOLDER LIST: 0 Minutes, 41 Seconds


In [11]:
surveys = []
print('GETTING SURVEY LIST: ' + getTime())
for surveyUrl in surveysUrl:
    surveyList = Flow.getResponse(surveyUrl)
    for survey in surveyList['surveys']:
        surveys.append(survey)
print('SURVEY IS POPULATED: ' + getTime())

GETTING SURVEY LIST: 0 Minutes, 45 Seconds
GETTING SURVEY LIST: 0 Minutes, 49 Seconds
GETTING SURVEY LIST: 0 Minutes, 54 Seconds
GETTING SURVEY LIST: 0 Minutes, 58 Seconds
GETTING SURVEY LIST: 1 Minutes, 0 Seconds
GETTING SURVEY LIST: 1 Minutes, 3 Seconds
GETTING SURVEY LIST: 1 Minutes, 7 Seconds
GETTING SURVEY LIST: 1 Minutes, 11 Seconds
GETTING SURVEY LIST: 1 Minutes, 15 Seconds
GETTING SURVEY LIST: 1 Minutes, 18 Seconds


In [12]:
for url in surveys:
    data = Flow.getResponse(url['surveyUrl'])
    print('GETTING {}: {}'.format(data['name'],getTime()))
    if data['registrationFormId'] == "":
        data.update({'registrationFormId':0})
    input_data = Surveys(data)
    writeData(input_data, data)
    for form in data['forms']:
        form.update({'survey_id':data['id']})
        input_data = Forms(form)
        writeData(input_data, form)
        for qgroup in form['questionGroups']:
            qgroup.update({'form_id':form['id']})
            input_data = QuestionGroups(qgroup)
            writeData(input_data, qgroup)
            for question in qgroup['questions']:
                question.update({'group_id':qgroup['id']})
                question.update({'form_id':form['id']})
                input_data = Questions(question)
                writeData(input_data, question)

GETTING Farmer Baseline: 1 Minutes, 23 Seconds
ERROR: ABORTING Farmer Baseline
ERROR: ABORTING Farmer Baseline
ERROR: ABORTING Farmer profile
ERROR: ABORTING Can we ask you a number of questions and use the information in the report for our project monitoring and share it to our donors ? Note:  -Introduce about the Enumerator’s name, role. -Debrief about Angkor Salad: is a project provides technical advise through mobile App, irrigation & fertilizer advice, crop planning, market information and Khmer GAP compliance on main 6 vegetable crops.
ERROR: ABORTING Location
ERROR: ABORTING Village Name
ERROR: ABORTING Interview Location
ERROR: ABORTING Photo of the Farm
ERROR: ABORTING Farmer Name
ERROR: ABORTING Age
ERROR: ABORTING Gender
ERROR: ABORTING Role in the family
ERROR: ABORTING What type of phone do you use ?
ERROR: ABORTING How many phone line do you use?
ERROR: ABORTING Can you use the phone from any member in your household when you need to make a call or check information on in

GETTING Collector details from farmer: 1 Minutes, 26 Seconds
ERROR: ABORTING Collector details from farmer
ERROR: ABORTING New form
ERROR: ABORTING Basic
ERROR: ABORTING Location of farmer
ERROR: ABORTING Name of farmer
ERROR: ABORTING Name of collector
ERROR: ABORTING Phone Number of collector (Fill 999 if not available)
ERROR: ABORTING Vegetables collected
ERROR: ABORTING Type of vegetable
GETTING Vegetable Collector survey: 1 Minutes, 30 Seconds
ERROR: ABORTING Vegetable Collector survey
ERROR: ABORTING Buying Details
ERROR: ABORTING Buying details
ERROR: ABORTING Type of crop
ERROR: ABORTING Approximate quantity collected every month in season (kg)
ERROR: ABORTING Collection Area
ERROR: ABORTING Collection communes
ERROR: ABORTING Collector Registration
ERROR: ABORTING Primary Details
ERROR: ABORTING Primary address of collector
ERROR: ABORTING Village
ERROR: ABORTING Name
ERROR: ABORTING Gender
ERROR: ABORTING Age
ERROR: ABORTING Contact phone number
ERROR: ABORTING Where do you s

GETTING Kinna's First Survey: 1 Minutes, 56 Seconds
ERROR: ABORTING Kinna's First Survey
ERROR: ABORTING New form
ERROR: ABORTING Primary Information
ERROR: ABORTING Name
GETTING New survey: 2 Minutes, 0 Seconds
ERROR: ABORTING New survey
ERROR: ABORTING New form
ERROR: ABORTING New group - please change name
GETTING Field Survey: 2 Minutes, 1 Seconds
ERROR: ABORTING Field Survey
ERROR: ABORTING Form 1
ERROR: ABORTING Identity
ERROR: ABORTING Name
ERROR: ABORTING Gender
ERROR: ABORTING Age
ERROR: ABORTING Have Smartphone
ERROR: ABORTING Monthly income (in thousand riel)
ERROR: ABORTING Monthly spending (in thousand riel)
ERROR: ABORTING Primary service need
ERROR: ABORTING Crops
ERROR: ABORTING Current Crop
ERROR: ABORTING Crop Area in current season (m2)
ERROR: ABORTING Last Season crop
ERROR: ABORTING Crops Area in the last season (m2)
ERROR: ABORTING Production last season (kg)
ERROR: ABORTING Water usage
ERROR: ABORTING Fertilizer usage
ERROR: ABORTING Type
ERROR: ABORTING Geolocat

In [13]:
session.rollback()

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/sqlalchemy/orm/session.py:561: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [18]:
total_surveys = session.query(Surveys).count()
total_forms = session.query(Forms).count()
total_question_groups = session.query(QuestionGroups).count()
total_questions = session.query(Questions).count()

In [26]:
print(
    'SUCCESS : {}\n\n'\
    '{} SURVEYS\n'\
    '{} FORMS\n'\
    '{} QUESTION GROUPS\n'\
    '{} QUESTIONS\n'\
    .format(getTime(),total_surveys,total_forms,total_question_groups,total_questions)
)

SUCCESS : 12 Minutes, 20 Seconds

17 SURVEYS
34 FORMS
58 QUESTION GROUPS
403 QUESTIONS

